# Initialization

Test notebook for the C-MAPPS benchmark. Test the Tunable Model class architecture

First we import the necessary packages and create the global variables.

In [2]:
import math
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

from scipy.optimize import differential_evolution

Using TensorFlow backend.


# Create models

Create different types of models to be tested with the Tunable Model class

In [3]:
#Clear the previous tensorflow graph
K.clear_session()

FilterN = 10
FilterL = 10

def RULmodel_SN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

def RULmodel_SN_2(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    #model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
    #                kernel_regularizer=regularizers.l2(0.1), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULModel_CNN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    print(input_shape)
    
    #Add the layers for the model
    model.add(Conv2D(1, (FilterN, FilterL), input_shape=input_shape, padding='same', kernel_initializer='glorot_normal', activation='relu', name='cl1'))
    model.add(Dropout(0.6))
    model.add(MaxPooling2D(pool_size=5, name='pl1'))
    model.add(Flatten())
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

# Load the data

In [4]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']

selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
#selected_indices = np.array([7, 8, 9, 12, 16, 17, 20])
#selected_indices = np.array(list(range(1,22)))

selected_features = list(features[i] for i in selected_indices-1)

print(selected_features)

windowSize = 30
stride = 2
constRUL = 125


dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_2(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, 'keras', window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
tModel.changeDataset(1)
tModel.loadData(verbose=1, rectify_labels = False)
tModel.printData()

['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 'htBleed', 'W31', 'W32']
Loading data for dataset 1 with window_size of 30, stride of 2 and constRUL of 125. Cros-Validation ratio 0
Data loaded for dataset 1
Printing shapes

Training data (X, y)
(8890, 420)
(8890,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.61403509 -0.00691857 -0.39391206 ..., -0.8         0.41463415
   0.4921528 ]
 [-0.24210526 -0.09792443 -0.23156708 ..., -0.4         0.36585366
   0.7554042 ]
 [-0.22807018 -0.37147419 -0.14047351 ..., -0.4         0.02439024
   0.19070181]
 [-0.15087719  0.1325173  -0.52153326 ..., -0.4         0.10569106
   0.55078472]
 [-0.40350877  0.06120277 -0.65546787 ..., -0.4         0.43089431
   0.6870003 ]]
[ 125.  125.  125.  125.  125.]
Testing data (X, y)
[[-0.69122807 -0.07344332 -0.62750846 ..., -0.2         0.02439024
   0.32691738]
 [ 0.01754386 -0.03618946 -0.38985344 ..., -0.6         0.02439024
   0.050

In [5]:
#tModel.printData(printTop=False)
#tModel.windowSize = 20
#tModel.changeDataset(2)
#tModel.loadData(verbose=1, crossValRatio=0.1)
#tModel.printData(printTop=False)
tModel.getModelDescription()
#ruldcnn = RULCNNModel(tModel.windowSize, len(selected_features))
#ruldcnn.summary()
#tModel.trimmedRUL_train
#display(tModel.df_train)

Description for model: ModelRUL_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 20)                8420      
_________________________________________________________________
out (Dense)                  (None, 1)                 21        
Total params: 8,441
Trainable params: 8,441
Non-trainable params: 0
_________________________________________________________________


# Train the model and test some Tunable Model functionalities

In [6]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
#lrate = None

#print(tModel.currentModelName)
#tModel.getAllModelsDescription()
#tModel.getModelDescription('RULModel_CNN_1')
#tModel.getModelDescription(tModel.currentModelName)
#tModel.getModelDescription()
#tModel.setCurrentModel('RULmodel_SN_1')
#tModel.getModelDescription()
#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_2(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
tModel.changeModel('ModelRUL_SN_1', modelRULSN, 'keras')
tModel.epochs = 100
tModel.trainModel(learningRateScheduler=lrate, verbose=1)

Epoch 1/100
8890/8890 [==============================] - 1s 74us/step - loss: 7909.9379 - mean_squared_error: 7909.9379
Epoch 2/100
8890/8890 [==============================] - 0s 13us/step - loss: 6961.4261 - mean_squared_error: 6961.4261
Epoch 3/100
8890/8890 [==============================] - 0s 14us/step - loss: 5996.7597 - mean_squared_error: 5996.7597
Epoch 4/100
8890/8890 [==============================] - 0s 14us/step - loss: 5001.0505 - mean_squared_error: 5001.0505
Epoch 5/100
8890/8890 [==============================] - 0s 13us/step - loss: 4097.5826 - mean_squared_error: 4097.5826
Epoch 6/100
8890/8890 [==============================] - 0s 12us/step - loss: 3333.6800 - mean_squared_error: 3333.6800
Epoch 7/100
8890/8890 [==============================] - 0s 13us/step - loss: 2731.9943 - mean_squared_error: 2731.9943
Epoch 8/100
8890/8890 [==============================] - 0s 13us/step - loss: 2286.9668 - mean_squared_error: 2286.9668
Epoch 9/100
8890/8890 [=================

In [7]:
tModel.evaluateModel(["rhs"], round=2, setLimits=[7,145])
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.trainTime

i = 1
for prediction in tModel.y_pred:
    print('Engine {}, RUL {}, Rounded RUL {}, Real RUL {}'.format(i, prediction, 
                                                                    tModel.y_pred_rectified[i-1], 
                                                                    tModel.y_test[i-1]))
    i = i+1

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 255us/step
limits
[  0.  16.  -8.   7.  11.  12.  15.  -5.   5. -10.  -5. -23. -10.   3.  26.
  20.   9.  -4.   9.  -9.  33.  13.   8. -13. -29.  -8.  38.  10.  14. -13.
  -1.   5.  -4.   0.   9.   4.   9.  11. -17.   3.  -4.  -3.  15.  -8. -34.
  -2. -26.  11. -11.  22. -11.  -7.   4.  16. -25.  10.  -4.  22. -12.   5.
 -14.  -7.   9.  -6.  -9.  -1.  48.  -1.  -5.  -6. -10.  23. -27. -23.  -5.
  -3.   8.  36.  35.   4.   1.  -2.  -9.  22.  15.  21.  -5.  -1. -27.  -3.
  -7.   7. -41.  16. -17. -43.   2.   8.  -6. -13.]
scores
Engine 1, RUL [ 112.09597015], Rounded RUL 112.0, Real RUL 112.0
Engine 2, RUL [ 114.98062897], Rounded RUL 114.0, Real RUL 98.0
Engine 3, RUL [ 61.13838577], Rounded RUL 61.0, Real RUL 69.0
Engine 4, RUL [ 89.6437912], Rounded RUL 89.0, Real RUL 82.0
Engine 5, RUL [ 102.12400055], Rounded RUL 102.0, Real RUL 91.0
Engine 6, RUL [ 105.61565399], Rounded RUL 105.0, Real RUL 93.0
Engine 7, RUL [ 106.07927704], Rounded RU

In [ ]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    windows_size = x[0]
    window_stride = x[1]
    constantRUL = x[2]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN_1(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData()
    
    #Add the models to the tunableModel object
    tModel.addModel("RUL_SN_1_optmizable", modelRULSN)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainCurrentModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.getModelTimes(tModel.currentModelName)
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateCurrentModel(["rhs"], crossValidation=True)
    cScores = tModel.getModelScores(tModel.currentModelName)
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse

In [ ]:
def plotRUL(cycles, rulArray, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    
    plt.show()
    
def plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, constRUL, stride, y_test):

    df = tModel.df_test
    df_engine = df[df['Unit Number'] == engineUnit]
    
    X_test2, _, _, _ = CMAPSAuxFunctions.create_windowed_data(df_engine, selected_features, 'test', time_window=time_window, 
                                                     constRUL=constRUL, stride=stride)
    
    X_test2 = tModel.dataScaler.transform(X_test2)
    nnPred = tModel.model.predict(X_test2)
    
    maxCycle = X_test2.shape[0]
    faultCycle = tModel.y_test[engineUnit-1]
    cycles = np.arange(maxCycle)
    rulArray = np.arange(faultCycle, maxCycle+faultCycle)
    rulArray[rulArray > constRUL] = constRUL
    rulArray = np.flipud(rulArray)
    
    plotRUL(cycles, rulArray, engineUnit)
    plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.legend()

In [ ]:
plotRUL_Engine(tModel, engineUnit, selectedFeatures, time_window, stride, constRUL)

In [17]:
l = np.array([  2,  16, -10,   6,   9,  11,  17,  -5,   4,  -9,  -5, -21, -10,   2,  27,
  22,  10,  -3,  11,  -9,  35,  13,  10, -13, -28,  -6,  39,   9,  16, -14,
  -1,   3,  -5,   0,   5,   4,   9,  12, -13,   2,  -4,  -3,  15,  -9, -34,
   0, -27,  11,  -9,  23, -11,  -6,   6,  17, -23,  12,  -4,  20, -11,   3,
 -14,  -6,   8,  -7,  -9,  -4,  50,  -1,  -5,  -6,  -8,  24, -26, -24,  -4,
  -3,   7,  36,  35,   3,  -1,  -2,  -7,  21,  14,  21,  -2,  -2, -27,  -4,
  -7,   7, -44,  16, -16, -45,   5,   8,  -9, -13])

print(l)
l2 = l**2
print(l2)
suma = np.sum(l2)
print(suma)
r2 = math.sqrt(suma/100)
print(r2)

[  2  16 -10   6   9  11  17  -5   4  -9  -5 -21 -10   2  27  22  10  -3
  11  -9  35  13  10 -13 -28  -6  39   9  16 -14  -1   3  -5   0   5   4
   9  12 -13   2  -4  -3  15  -9 -34   0 -27  11  -9  23 -11  -6   6  17
 -23  12  -4  20 -11   3 -14  -6   8  -7  -9  -4  50  -1  -5  -6  -8  24
 -26 -24  -4  -3   7  36  35   3  -1  -2  -7  21  14  21  -2  -2 -27  -4
  -7   7 -44  16 -16 -45   5   8  -9 -13]
[   4  256  100   36   81  121  289   25   16   81   25  441  100    4  729
  484  100    9  121   81 1225  169  100  169  784   36 1521   81  256  196
    1    9   25    0   25   16   81  144  169    4   16    9  225   81 1156
    0  729  121   81  529  121   36   36  289  529  144   16  400  121    9
  196   36   64   49   81   16 2500    1   25   36   64  576  676  576   16
    9   49 1296 1225    9    1    4   49  441  196  441    4    4  729   16
   49   49 1936  256  256 2025   25   64   81  169]
27057
16.44901212839239
